In [71]:
import pandas as pd
import re
from tqdm import tqdm

In [72]:
recipe_df = pd.read_csv("kr_recipe.csv")

In [ ]:
# 열 이름 변경
recipe_df.rename(columns={'RCP_SNO': '레시피일련번호', 'RCP_TTL': '레시피제목', 'CKG_NM': '요리명',
                          'RGTR_ID': '등록자ID', 'RGTR_NM': '등록자명', 'INQ_CNT': '조회수',
                          'RCMM_CNT': '추천수', 'SRAP_CNT': '스크랩수', 'CKG_MTH_ACTO_NM': '요리타입',
                          'CKG_STA_ACTO_NM': '음식분위기', 'CKG_MTRL_ACTO_NM': '재료타입', 'CKG_KND_ACTO_NM': '음식타입',
                          'CKG_IPDC': '요리소개', 'CKG_MTRL_CN': '재료리스트', 'CKG_INBUN_NM': '몇인분',
                          'CKG_DODF_NM': '요리난이도', 'CKG_TIME_NM': '요리시간', 'FIRST_REG_DT': '최초등록일시'}, inplace=True)

In [ ]:
# 불필요한 열 제거
recipe_df = recipe_df.drop(columns=['레시피일련번호', '레시피제목', '등록자ID', '등록자명', '요리소개', '최초등록일시'])

# 다음 열들에서 결측치 제거
recipe_df = recipe_df.dropna(subset=['재료리스트', '요리명'])

In [ ]:
# 다음 열들에서 데이터 수정
modify_col_data = ['추천수', '스크랩수']

for col in modify_col_data:
    
    # 숫자로 변환할 수 없는 값 NaN으로 변경
    recipe_df[col] = pd.to_numeric(recipe_df[col], errors='coerce')
    
    # NaN => 0
    recipe_df[col].fillna(0, inplace=True)
    
    # object => int64
    recipe_df[col] = recipe_df[col].astype('int64')

In [74]:
recipe_df

,요리명,조회수,추천수,스크랩수,요리타입,음식분위기,재료타입,음식타입,재료리스트,몇인분,요리난이도,요리시간
0,어묵김말이,10072,6,66,튀김,간식,가공식품류,디저트,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T,2인분,초급,60분이내
1,두부새우전,5817,3,27,부침,일상,해물류,밑반찬,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개,3인분,초급,30분이내
2,알밥,6975,8,36,굽기,일상,해물류,밥/죽/떡,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무순 약간| 날치알 6스푼| 김가루 약간| 후리가케(또는밥이랑같은류)| 참기름 약간| 통깨 약간| 계란 노른자 2알,2인분,초급,30분이내
3,현미호두죽,3339,0,11,끓이기,일상,쌀,밥/죽/떡,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내
4,북어갈비,7173,3,97,굽기,술안주,건어물류,메인반찬,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T| 다진파 1T| 다진마늘 1T| 참기름 1T| 깨소금 1T| 후춧가루 약간,2인분,초급,60분이내
...,...,...,...,...,...,...,...,...,...,...,...,...
183919,명란젓크림스파게티,59,0,0,기타,손님접대,해물류,양식,[재료] 올리브오일| 스파게티면 80g| 명란젓 1덩이| 생크림 100g| 마늘 1~2톨| 양파 1/4개| 소금 약간| 후추 약간,1인분,아무나,NaN
183920,아마트리치아나스파게티,46,0,0,기타,손님접대,돼지고기,양식,[재료] 올리브오일| 스파게티면 280~300g| 관찰레 120g| 홀토마토 1캔| 화이트와인 50ml| 파르미지아노 치즈 100g| 소금 약간,4인분,아무나,30분이내
183921,오징어젓무침,110,0,8,무침,일상,가공식품류,김치/젓갈/장류,[재료] 오징어젓200g| 무말랭이15g| 청양고추2개| 홍고추1개| 오이고추1개| 쪽파 2줄| 양파 1/4개| 다진마늘 1T| 통깨 1T| 참기름 1T,2인분,아무나,NaN
183922,계란찜,170,0,6,찜,일상,달걀/유제품,밑반찬,[재료] 달걀 6개| 우유 200ml| 물 100ml| 맛살 3개| 당근 1/3개| 대파 1/3대 [양념] 소금 1/3| 참치액 1/2| 맛술 1/2,3인분,아무나,30분이내


In [75]:
def extraction_of_nouns(sentence):
    # 패턴
    pattern = r'\[\w+\]' # ex) [재료], [양념]

    # 패턴으로 분리
    parts = re.split(pattern, sentence)

    # 각 파트에서 양쪽 여백 제거, 비어있지 않은 리스트만 추출
    parts = [list(map(lambda x: x.strip(), part.split("|"))) for part in parts if part]

    # 2차원 리스트 평탄화
    flattend_parts = [item for part in parts for item in part]

    pattern = r'^[^\d\(\)]+' # 숫자와 괄호로 시작하지 않는 패턴

    result = []
    for part in flattend_parts:
        matches = re.findall(pattern, part) # 매치 결과
        for match in matches:
            result.append(match.strip()) # 양쪽 여백 제거후 추가하기
    
    return result

# 진행 상황을 출력하는 헬퍼 함수
def progress_apply(series, func):
    result = []
    for i, item in enumerate(tqdm(series, desc="Processing", unit="row")):
        result.append(func(item))
    return result

In [76]:
# 새로운 열 추가
# recipe_df['추출된명사'] = recipe_df['재료리스트'].apply(extraction_of_nouns)
recipe_df['추출된명사'] = progress_apply(recipe_df['재료리스트'], extraction_of_nouns)

Processing: 100%|██████████| 183924/183924 [00:08<00:00, 22864.48row/s]


In [77]:
recipe_df

,요리명,조회수,추천수,스크랩수,요리타입,음식분위기,재료타입,음식타입,재료리스트,몇인분,요리난이도,요리시간,추출된명사
0,어묵김말이,10072,6,66,튀김,간식,가공식품류,디저트,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T,2인분,초급,60분이내,"[어묵, 김밥용김, 당면, 양파, 당근, 깻잎, 튀김가루, 올리브유 적당량, 간장, 참기름]"
1,두부새우전,5817,3,27,부침,일상,해물류,밑반찬,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개,3인분,초급,30분이내,"[두부, 당근, 고추, 브로콜리, 새우, 녹말가루, 계란]"
2,알밥,6975,8,36,굽기,일상,해물류,밥/죽/떡,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무순 약간| 날치알 6스푼| 김가루 약간| 후리가케(또는밥이랑같은류)| 참기름 약간| 통깨 약간| 계란 노른자 2알,2인분,초급,30분이내,"[밥, 당근, 치자단무지, 신김치, 무순 약간, 날치알, 김가루 약간, 후리가케, 참기름 약간, 통깨 약간, 계란 노른자]"
3,현미호두죽,3339,0,11,끓이기,일상,쌀,밥/죽/떡,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내,"[현미, 찹쌀, 호두, 물, 소금 약간]"
4,북어갈비,7173,3,97,굽기,술안주,건어물류,메인반찬,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T| 다진파 1T| 다진마늘 1T| 참기름 1T| 깨소금 1T| 후춧가루 약간,2인분,초급,60분이내,"[북어포, 찹쌀가루, 간장, 설탕, 물, 다진파, 다진마늘, 참기름, 깨소금, 후춧가루 약간]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183919,명란젓크림스파게티,59,0,0,기타,손님접대,해물류,양식,[재료] 올리브오일| 스파게티면 80g| 명란젓 1덩이| 생크림 100g| 마늘 1~2톨| 양파 1/4개| 소금 약간| 후추 약간,1인분,아무나,NaN,"[올리브오일, 스파게티면, 명란젓, 생크림, 마늘, 양파, 소금 약간, 후추 약간]"
183920,아마트리치아나스파게티,46,0,0,기타,손님접대,돼지고기,양식,[재료] 올리브오일| 스파게티면 280~300g| 관찰레 120g| 홀토마토 1캔| 화이트와인 50ml| 파르미지아노 치즈 100g| 소금 약간,4인분,아무나,30분이내,"[올리브오일, 스파게티면, 관찰레, 홀토마토, 화이트와인, 파르미지아노 치즈, 소금 약간]"
183921,오징어젓무침,110,0,8,무침,일상,가공식품류,김치/젓갈/장류,[재료] 오징어젓200g| 무말랭이15g| 청양고추2개| 홍고추1개| 오이고추1개| 쪽파 2줄| 양파 1/4개| 다진마늘 1T| 통깨 1T| 참기름 1T,2인분,아무나,NaN,"[오징어젓, 무말랭이, 청양고추, 홍고추, 오이고추, 쪽파, 양파, 다진마늘, 통깨, 참기름]"
183922,계란찜,170,0,6,찜,일상,달걀/유제품,밑반찬,[재료] 달걀 6개| 우유 200ml| 물 100ml| 맛살 3개| 당근 1/3개| 대파 1/3대 [양념] 소금 1/3| 참치액 1/2| 맛술 1/2,3인분,아무나,30분이내,"[달걀, 우유, 물, 맛살, 당근, 대파, 소금, 참치액, 맛술]"


In [78]:
# 전처리된 데이터프레임 저장
recipe_df.to_csv('kr_recipe.csv', index=False, encoding='utf-8-sig')